# Логистическая регрессия

Обучим логистическую регрессию для предсказания того, откликнется клиент  
на рекламное предложение (target = 1) или нет (target = 0).

## Подлючение библиотек, загрузка и обзор данных

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [ ]:
data = pd.read_csv('https://raw.githubusercontent.com/evgpat/edu_stepik_practical_ml/main/datasets/clients_data.csv')

In [ ]:
data.head()

,AGE,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,GENDER,CHILD_TOTAL,DEPENDANTS,PERSONAL_INCOME,LOAN_NUM_TOTAL,LOAN_NUM_CLOSED,LOAN_DLQ_NUM,TARGET
0,49,1,0,1,2,1,5000.0,1,1,2,0
1,32,1,0,1,3,3,12000.0,1,1,1,0
2,52,1,0,1,4,0,9000.0,2,1,0,0
3,39,1,0,1,1,1,25000.0,1,1,3,0
4,30,1,0,0,0,0,12000.0,2,1,2,0


In [ ]:
from sklearn.model_selection import train_test_split

X = data.drop('TARGET', axis=1)
y = data['TARGET']

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, train_size=0.7, random_state=123)

## Практика

**Задание**  
Выведите на экран количество объектов каждого класса. Сколько процентов объектов относятся к положительному классу?  
Ответ округлите до целого числа (например, если доля объектов положительного класса равна 0.412, в ответ запишите 41,  
имея в виду 41 процент).

In [ ]:
# ваш код здесь
y.value_counts()

,count
TARGET,
0,13411
1,1812


In [ ]:
y.value_counts(normalize=True).round(2) * 100

,proportion
TARGET,
0,88.0
1,12.0


In [ ]:
round(1812/13411*100)


14

Обучим логистическую регрессию с параметрами по умолчанию.

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()

In [ ]:
lr.fit(Xtrain, ytrain)

prediction = lr.predict(Xtest)

Метрику accuracy не стоит использовать при сильном дисбалансе классов. Поэтому посчитайте f1_score для оценки качества модели на тестовых данных.

f1_score принимает значения от 0 до 1. Чем ближе к 1, тем лучше модель.

In [ ]:
from sklearn.metrics import f1_score


# Оценка модели

f1_lr = f1_score(ytest, prediction)


print(f'f1: {f1_lr}')
print(round(f1_lr, 2))

f1: 0.0
0.0


**Вопрос**  
Чему равен `f1_score`?

Удивительно, да?

Давайте разберемся, почему качество такое низкое.

Предскажем вероятности классов с помощью обученной логистической регрессии на тестовых данных.

In [ ]:
probs_test = lr.predict_proba(Xtest)

probs_test[:10]

array([[0.86331972, 0.13668028],
       [0.86305957, 0.13694043],
       [0.80967732, 0.19032268],
       [0.73110167, 0.26889833],
       [0.87780414, 0.12219586],
       [0.96122864, 0.03877136],
       [0.95965491, 0.04034509],
       [0.96560659, 0.03439341],
       [0.92114871, 0.07885129],
       [0.9349701 , 0.0650299 ]])

По вероятностям видно, что вероятности отнесения к положительному классу очень низкие.

Попробуем изменить порог для перевода вероятности в классы.

**Вопрос**  
Чему равен `f1_score`, если все объекты с вероятностью не меньшей 0.1, относить к положительному классу?

Ответ округлите до сотых.

In [ ]:
prediction[:100]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
probs_lr = lr.predict_proba(Xtest) # нечестные вероятности

probs_lr[:,1]

array([0.13668028, 0.13694043, 0.19032268, ..., 0.09132017, 0.09727959,
       0.07518077])

In [ ]:
# ваш код здесь
lr_classes = probs_lr[:,1] >= 0.1

f1_score(ytest, lr_classes)


0.2469521725539231

У обученной модели можно посмотреть веса (как и в линейной регрессии).  
Выведем на экран веса модели (`model.coef_`, `model.intercept_`).

In [ ]:
lr.coef_, lr.intercept_

(array([[-5.55214705e-02, -7.90868782e-04, -4.28172545e-04,
         -1.07248154e-03, -1.34958177e-03, -4.95807875e-04,
          1.13131722e-05, -1.80011616e-03, -1.26843809e-03,
         -1.45283567e-05]]),
 array([-0.00115086]))

Создайте `pd.DataFrame`, где в первом столбце стоят названия признаков, а во втором - их веса (так удобнее анализировать результат).  
Отсортируйте таблицу по убыванию весов.

**Вопрос**  
Какой признак имеет наибольший положительный вес?

In [ ]:
# ваш код здесь
importances = pd.DataFrame({'weights': lr.coef_[0], 'features': Xtrain.columns}).sort_values(by='weights')
importances

,weights,features
0,-0.055521,AGE
7,-0.001800,LOAN_NUM_TOTAL
4,-0.001350,CHILD_TOTAL
8,-0.001268,LOAN_NUM_CLOSED
3,-0.001072,GENDER
1,-0.000791,SOCSTATUS_WORK_FL
5,-0.000496,DEPENDANTS
2,-0.000428,SOCSTATUS_PENS_FL
9,-0.000015,LOAN_DLQ_NUM
6,0.000011,PERSONAL_INCOME


In [ ]:
features_weights = {
    'features' : data.columns[:-1],
    'weights' : lr.coef_[0]
    }
pd.DataFrame(data=features_weights).sort_values(by=['weights'], ascending=False)

,features,weights
6,PERSONAL_INCOME,0.000011
9,LOAN_DLQ_NUM,-0.000015
2,SOCSTATUS_PENS_FL,-0.000428
5,DEPENDANTS,-0.000496
1,SOCSTATUS_WORK_FL,-0.000791
3,GENDER,-0.001072
8,LOAN_NUM_CLOSED,-0.001268
4,CHILD_TOTAL,-0.001350
7,LOAN_NUM_TOTAL,-0.001800
0,AGE,-0.055521


In [ ]:
coef = pd.DataFrame(index = X.columns, data = lr.coef_[0], columns = ['coef'])

coef.sort_values('coef', ascending = False)

,coef
PERSONAL_INCOME,0.000011
LOAN_DLQ_NUM,-0.000015
SOCSTATUS_PENS_FL,-0.000428
DEPENDANTS,-0.000496
SOCSTATUS_WORK_FL,-0.000791
GENDER,-0.001072
LOAN_NUM_CLOSED,-0.001268
CHILD_TOTAL,-0.001350
LOAN_NUM_TOTAL,-0.001800
AGE,-0.055521


## Бонус

**Задание 1**  
Подберите порог для перевода вероятностей в классы, дающий максимальное значение `f1_score`.

In [33]:
# ваш код здесь
k = 0.0

while k < 1:

  lr_classes_K = probs_lr[:,1] >= k
  print('k = ', k, '        f1= ', f1_score(ytest, lr_classes_K))
  k += 0.05

k =  0.0         f1=  0.21810378462738977
k =  0.05         f1=  0.23141967067200714
k =  0.1         f1=  0.2469521725539231
k =  0.15000000000000002         f1=  0.236745886654479
k =  0.2         f1=  0.20736288504883546
k =  0.25         f1=  0.07564296520423601
k =  0.3         f1=  0.01061946902654867
k =  0.35         f1=  0.0
k =  0.39999999999999997         f1=  0.0
k =  0.44999999999999996         f1=  0.0
k =  0.49999999999999994         f1=  0.0
k =  0.5499999999999999         f1=  0.0
k =  0.6         f1=  0.0
k =  0.65         f1=  0.0
k =  0.7000000000000001         f1=  0.0
k =  0.7500000000000001         f1=  0.0
k =  0.8000000000000002         f1=  0.0
k =  0.8500000000000002         f1=  0.0
k =  0.9000000000000002         f1=  0.0
k =  0.9500000000000003         f1=  0.0


**Задание 2**  
Во вложенном цикле подберите одновременно коэффициент регуляризации `C` у логистической регрессии и порог для перевода вероятностей в классы, дающие максимальное значение `f1_score`.

По-хорошему, чтобы не переобучиться, эти величины надо подбирать не по тесту, а по отдельной выборке.

Поэтому разобъем данные изначально на три части: `Xtrain`, `Xval`, `Xtest`.

*   В цикле при подборе `С` и порога будем обучаться по `Xtrain`, а предсказывать и измерять качество по `Xval`.

*   Качество итоговой модели с найденными `C` и порогом измерьте по `Xtest`.

Так не переобучимся!


In [35]:
# your code here

from sklearn.model_selection import train_test_split


df = pd.read_csv('https://raw.githubusercontent.com/evgpat/edu_stepik_practical_ml/main/datasets/clients_data.csv')

X = df.drop('TARGET', axis=1)
y = df['TARGET']

X_train, X_val, y_train, y_val = train_test_split(X , y, train_size=0.7, random_state=123)

In [36]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

max_precision = -1
limit = 0
i = 0

model =LogisticRegression()
model.fit(X_train, y_train)
pred_test = model.predict(X_val)

prob = model.predict_proba(X_val)
prob_churn = prob[:,1]

while i <=1:
  classes = prob_churn > i
  if (f1_score(y_val, classes) > max_precision):
    limit = i
    max_precision = precision_score(y_val, classes)
  i += 0.01

In [37]:
limit, max_precision

(0.21000000000000005, 0.17733990147783252)

In [39]:
# your code here
classes = prob_churn > limit

print(f"Accuracy: {accuracy_score(y_val, classes)}")
print(f"Precision: {precision_score(y_val, classes)}")
print(f"Recall: {recall_score(y_val, classes)}")
print(f"f1: {f1_score(y_val, classes)}")

Accuracy: 0.791548062185242
Precision: 0.17733990147783252
Recall: 0.19320214669051877
f1: 0.18493150684931506


In [ ]:
#Xtrain_new, Xval, ytrain_new, yval = train_test_split(Xtrain, ytrain, train_size=0.7, random_state=123)

# ваш код для подбора C и порога здесь

In [ ]:
model = LogisticRegression(C = ...)

model.fit(Xtrain, ytrain) # обучаемся на всех тренировочных данных

prediction = model.predict_proba(Xtest)

classes = ... # переведите полученные вероятности в классы по найденному порогу

In [ ]:
# вычислите значение f1_score на тестовых данных